<a href="https://colab.research.google.com/github/aritanaoya/football_analysis/blob/main/J1LeagueScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import Module

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen
from contextlib import closing
from operator import attrgetter
from pprint import pprint
from subprocess import DEVNULL
from subprocess import Popen
import time


# Team Dic

In [2]:
TeamDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126', '名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}

tmpDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126','名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}

tmp2Dic = {'清水':'126', '福岡':'135'}

J12021TeamDic = {'札幌':'sapp', '仙台':'send', '鹿島':'kasm', '浦和':'uraw', '柏':'kasw', 'FC東京':'fctk', '川崎Ｆ':'ka-f', '横浜FM':'y-fm', '横浜FC':'y-fc' , '湘南':'shon', '清水':'shim', '名古屋':'nago', 'Ｇ大阪':'g-os','Ｃ大阪':'c-os', '神戸':'kobe','広島':'hiro', '徳島':'toku', '福岡':'fuku', '鳥栖':'tosu', '大分':'oita'}

def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

get_keys_from_value(TeamDic,'276')

['札幌']

<select id="comparison1_team"><option value="0">チーム選択</option><option value="276">札幌</option><option value="31219">八戸</option><option value="30673">岩手</option><option value="238">仙台</option><option value="30310">秋田</option><option value="294">山形</option><option value="30674">福島</option><option value="120">鹿島</option><option value="176">水戸</option><option value="30305">栃木</option><option value="30103">群馬</option><option value="122">浦和</option><option value="199">大宮</option><option value="121">千葉</option><option value="132">柏</option><option value="270">FC東京</option><option value="123">東京Ｖ</option><option value="30532">町田</option><option value="86">川崎Ｆ</option><option value="124">横浜FM</option><option value="296">横浜FC</option><option value="30675">YS横浜</option><option value="130">湘南</option><option value="30676">相模原</option><option value="150">甲府</option><option value="30528">松本</option><option value="30535">長野</option><option value="193">新潟</option><option value="30308">富山</option><option value="30526">金沢</option><option value="126">清水</option><option value="131">磐田</option><option value="30677">藤枝</option><option value="31296">沼津</option><option value="127">名古屋</option><option value="30302">岐阜</option><option value="134">京都</option><option value="128">Ｇ大阪</option><option value="133">Ｃ大阪</option><option value="136">神戸</option><option value="30000">鳥取</option><option value="30314">岡山</option><option value="129">広島</option><option value="30851">山口</option><option value="30536">讃岐</option><option value="30116">徳島</option><option value="30148">愛媛</option><option value="30974">今治</option><option value="135">福岡</option><option value="30313">北九州</option><option value="269">鳥栖</option><option value="30531">長崎</option><option value="30303">熊本</option><option value="207">大分</option><option value="31508">宮崎</option><option value="31042">鹿児島</option><option value="30317">琉球</option></select>

# ScrapeData

In [ ]:
def scrapeData(homeTeamID,awayTeamID,year):
    url = 'https://www.football-lab.jp/comparison/team/' + year + '/' + homeTeamID + '/' + year + '/' + awayTeamID
    #チーム毎の平均スタッツを取得
    #チャンスビルディングポイント
    df = pd.read_html(url)[12]
    df = df.drop([0,2,4,6,8,10,12,14,16]).drop(df.columns[[0,1,5,6]], axis=1)
    # #チームスタイル
    #指数
    # df2 = pd.read_html(url)[13]
    # df2 = df2.drop([0,2,4,6,8,10,12,14]).drop(df2.columns[[0,1,5,6]], axis=1)
    #チャンス構築率
    df3 = pd.read_html(url)[16]
    df3 = df3.drop([0,2,4,6,8,10,11,13,15,17,19]).drop(df3.columns[[0,1,5,6]], axis=1)
    #ゴール期待値
    df4 = pd.read_html(url)[17]
    df4 = df4.drop([0,2,4,6]).drop(df4.columns[[0,1,5,6]], axis=1)
    #トラッキングデータ
    df5 = pd.read_html(url)[20]
    df5 = df5.drop([0,2,4,6]).drop(df5.columns[[0,1,5,6]], axis=1)

    #他のスタッツ
    df6 = pd.read_html(url)[35]
    df6 = df6.drop([0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34]).drop(df6.columns[[0,1,5,6]], axis=1)

    df = pd.concat([df,df3,df4,df5,df6])
    df = df.reset_index(drop=True)
    df = df.rename(columns={'試合平均': 'home','Unnamed: 3': 'Data','試合平均.1': 'away'})
    df = df[['Data', 'home', 'away']]
    df = df.set_index('Data')
    df = df.T

    time.sleep(1)

    #試合結果を取得
    html = requests.get(url)

    html.encoding = "UTF-8"
    soup = BeautifulSoup(html.text, "html.parser")

    texts = soup.find('div', attrs={'class': 'gameTbl'}).text
    texts = re.findall(r'\w+', texts)
    time.sleep(1) 

    return df,texts

# Scrape Season Data

In [3]:
def scrapeData(TeamName):
    url = 'https://www.football-lab.jp/' +  TeamName + '/match/'
    df = pd.read_html(url)[0]

    return df

In [4]:
df = scrapeData(J12021TeamDic['川崎Ｆ'])
df

,節,開催日,Unnamed: 2,相手,スコア,Unnamed: 5,会場,観客数,天候,AGI,KAGI,チャンス構築率,シュート,シュート成功率,支配率,攻撃CBP,パスCBP,奪取P,守備P,得点者,指揮官
0,1,2.26,(金),横浜FM,2-0,H,等々力,4868,曇,57,44,13.7%,18,11.1%,46.9%,15.98,14.30,139.34,13.68,家長x2,鬼木 達
1,11,3.3,(水),Ｃ大阪,3-2,H,等々力,4756,晴,67,61,16.4%,18,16.7%,65.0%,33.06,27.86,82.54,8.02,"Ｌダミアンx2,三笘",鬼木 達
2,2,3.6,(土),仙台,5-1,A,ユアスタ,9005,晴,57,47,11.7%,17,23.5%,59.7%,22.77,16.43,145.65,12.12,"遠野,小林x2,旗手,OG",鬼木 達
3,3,3.10,(水),徳島,2-0,H,等々力,4853,晴,59,71,16.2%,18,11.1%,49.2%,18.99,16.90,128.76,7.54,Ｌダミアンx2,鬼木 達
4,4,3.13,(土),柏,1-0,H,等々力,4744,雨,50,55,10.2%,14,7.1%,57.8%,20.39,16.04,105.05,17.13,家長,鬼木 達
5,5,3.17,(水),神戸,1-1,A,ノエスタ,7446,屋内,56,46,8.6%,13,7.7%,50.5%,19.37,16.15,128.84,16.96,Ｌダミアン,鬼木 達
6,6,3.21,(日),浦和,5-0,A,埼玉,4679,雨,60,67,13.9%,15,33.3%,44.3%,17.78,14.38,134.08,9.38,"Ｌダミアン,旗手,小林x2,脇坂",鬼木 達
7,7,4.3,(土),大分,2-0,H,等々力,9810,曇,66,85,15.7%,17,11.8%,48.0%,26.28,24.02,94.91,11.17,三笘x2,鬼木 達
8,8,4.7,(水),鳥栖,1-0,H,等々力,9173,晴,66,68,8.3%,10,10.0%,52.7%,22.66,19.66,132.41,9.52,遠野,鬼木 達
9,9,4.11,(日),FC東京,4-2,A,味スタ,17615,晴,61,53,10.3%,13,30.8%,54.7%,21.87,19.75,116.98,11.36,"家長x2,三笘,Ｌダミアン",鬼木 達
